In [2]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.5'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [69.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,378 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,771 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archiv

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as F

In [6]:
import json
tmdb_df = spark.read.json("movie_results.json")
tmdb_df.show()


NameError: name 'spark' is not defined

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("MoviesData").getOrCreate()

# Load the JSON file into a PySpark DataFrame
df = spark.read.json("movie_results.json")  # Replace with your JSON file path

# Select only the minimal columns: movieId, name, and popularity
df_selected = df.select(
    col("id").alias("movieId"),  # Renaming id to movieId
    col("title").alias("name"),  # Renaming title to name
    col("popularity")           # Keep popularity as is
)

# Show the result to verify
df_selected.show(truncate=False)


+-------+--------------------------------+----------+
|movieId|name                            |popularity|
+-------+--------------------------------+----------+
|950396 |The Gorge                       |210.407   |
|1126166|Flight Risk                     |199.997   |
|1064213|Anora                           |161.432   |
|762509 |Mufasa: The Lion King           |140.813   |
|1241982|Moana 2                         |137.318   |
|939243 |Sonic the Hedgehog 3            |78.835    |
|822119 |Captain America: Brave New World|66.759    |
|823219 |Flow                            |104.294   |
|927342 |Amaran                          |98.225    |
|426889 |Le Clitoris                     |52.914    |
|696506 |Mickey 17                       |85.297    |
|926670 |Henry Danger: The Movie         |49.298    |
|1084199|Companion                       |76.824    |
|1160956|Panda Plan                      |43.516    |
|912649 |Venom: The Last Dance           |65.533    |
|539972 |Kraven the Hunter  

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
# Load CSV files
rating_df = spark.read.csv('ratings.csv', header=True, inferSchema=True)  # Replace with the correct path
links_df = spark.read.csv('links.csv', header=True, inferSchema=True)    # Replace with the correct path

# Step 1: Join the links_df with df_selected to add tmdbId
df_with_tmdb = df_selected.join(links_df, 'movieId', 'inner')

# Step 2: Join the resulting DataFrame with rating_df on 'movieId' to include ratings
final_df = df_with_tmdb.join(rating_df, 'movieId', 'inner')

# Step 3: Select the minimal required columns for final output
final_df_selected = final_df.select(
    col("userId"),    # Keep userId
    col("tmdbId"),    # Keep tmdbId
    col("name"),      # Keep name (movie title)
    col("popularity"),  # Keep popularity
    col("rating")     # Keep rating
)

# Show the final result
final_df_selected.show(truncate=False)




+------+------+-----------------------+----------+------+
|userId|tmdbId|name                   |popularity|rating|
+------+------+-----------------------+----------+------+
|603   |8447  |Raiders of the Lost Ark|15.823    |5.0   |
|385   |8447  |Raiders of the Lost Ark|15.823    |3.0   |
|262   |8447  |Raiders of the Lost Ark|15.823    |3.0   |
|199   |8447  |Raiders of the Lost Ark|15.823    |1.0   |
|191   |8447  |Raiders of the Lost Ark|15.823    |5.0   |
|160   |8447  |Raiders of the Lost Ark|15.823    |5.0   |
|90    |8447  |Raiders of the Lost Ark|15.823    |5.0   |
|314   |36344 |Shrek                  |14.761    |3.0   |
|27    |36344 |Shrek                  |14.761    |4.0   |
|608   |114   |Titanic                |10.837    |2.5   |
|606   |114   |Titanic                |10.837    |4.0   |
|603   |114   |Titanic                |10.837    |3.0   |
|602   |114   |Titanic                |10.837    |2.0   |
|599   |114   |Titanic                |10.837    |3.0   |
|597   |114   

In [9]:
# We need to make sure the rating values are in a numerical format and handle any nulls if present.
final_df_selected = final_df_selected.dropna(subset=['rating'])  # Drop rows with missing ratings

# Step 2: Split data into training and test sets
(training_data, test_data) = final_df_selected.randomSplit([0.8, 0.2], seed=1234)


In [10]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
# Set up ALS with basic configuration
als = ALS(userCol="userId", itemCol="tmdbId", ratingCol="rating", coldStartStrategy="drop", nonnegative=True)

# Build a parameter grid for tuning
param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [10, 20, 30]) \
    .addGrid(als.maxIter, [5, 10, 15]) \
    .addGrid(als.regParam, [0.1, 0.2, 0.3]) \
    .build()

# Set up CrossValidator for better model evaluation
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
crossval = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# Run cross-validation and find the best model
cv_model = crossval.fit(final_df)


In [11]:
# Get best model's RMSE
best_model = cv_model.bestModel

# Retrieve parameters
rank = best_model.rank  # ALSModel stores rank as an attribute
max_iter = best_model._java_obj.parent().getMaxIter()  # Accessing maxIter from the parent estimator
reg_param = best_model._java_obj.parent().getRegParam()  # Accessing regParam from the parent estimator

print(f"Best Model's Parameters: rank={rank}, maxIter={max_iter}, regParam={reg_param}")



Best Model's Parameters: rank=10, maxIter=15, regParam=0.2


In [12]:
# Evaluate the best model
predictions = best_model.transform(test_data)
rmse = evaluator.evaluate(predictions)
print(f"RMSE of the best model: {rmse}")

RMSE of the best model: 0.7180592624312921
